

# Example: Train SimCLR on CIFAR10

In this tutorial, we will train a SimCLR model using lightly. The model,
augmentations and training procedure is from 
`A Simple Framework for Contrastive Learning of Visual Representations <https://arxiv.org/abs/2002.05709>`_.

The paper explores a rather simple training procedure for contrastive learning.


In [1]:
!nvidia-smi

Wed Mar 23 00:26:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla M60           Off  | 00000000:3D:00.0 Off |                  Off |
| N/A   29C    P0    37W / 150W |   1175MiB /  8129MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Imports

Import the Python frameworks we need for this tutorial.



In [2]:
import torch
import torch.nn as nn
import torchvision
import pytorch_lightning as pl
import lightly

from lightly.data import LightlyDataset
from torchvision.datasets import CIFAR10
from torch.utils.data import Subset
import sys


sys.path.append('../src')
from utils import get_classes, generate_embeddings, custom_collate_fn
from my_resnet import resnet20
%matplotlib inline

## Configuration

We set some configuration parameters for our experiment.
Feel free to change them and analyze the effect.



In [3]:
num_workers = 2
batch_size = 128
seed = 1

max_epochs = 150
input_size = 32 # image height, assume its always square

# Let's set the seed for our experiments
data_path = "../data/cifar10"
NORMAL_CLASS = 'dog'


pl.seed_everything(seed)

Global seed set to 1


1

## Setup data augmentations and loaders


In [4]:
cifar10_train = CIFAR10(data_path, download=True, train=True)
cifar10_test = CIFAR10(data_path, download=False, train=False)

classes_ids_train = get_classes(cifar10_train) # long!
classes_ids_test = get_classes(cifar10_test)

Files already downloaded and verified


`collate_fn` Additional augmentations such as vertical flip or random rotation (90 degrees). 
By adding these augmentations we learn our model invariance regarding the 
orientation of the images. E.g. we don't care if a shirt is upside down
but more about the strcture which make it.

In [5]:
dataset_train = LightlyDataset.from_torch_dataset(Subset(cifar10_train, classes_ids_train[NORMAL_CLASS]))
dataset_test = LightlyDataset.from_torch_dataset(Subset(cifar10_train, classes_ids_train[NORMAL_CLASS]))

dataloader_train = torch.utils.data.DataLoader(dataset_train,
    batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn, drop_last=True, num_workers=num_workers)

dataloader_test = torch.utils.data.DataLoader(dataset_test,
                                              batch_size=batch_size, shuffle=False, drop_last=False, collate_fn=custom_collate_fn, num_workers=num_workers)

# 1. Create the SimCLR Model
Now we create the SimCLR model. We implement it as a PyTorch Lightning Module
and use custom ResNet-20 backbone provided by Nikita Balabin. Lightly provides implementations
of the SimCLR projection head and loss function in the `SimCLRProjectionHead`
and `NTXentLoss` classes. We can simply import them and combine the building
blocks in the module. We will import constructed model from our `src`.



In [6]:
from simclr_model import SimCLRModel

We first check if a GPU is available and then train the module
using the PyTorch Lightning Trainer.



In [7]:
gpus = 1 if torch.cuda.is_available() else 0
device = "cuda" if torch.cuda.is_available() else "cpu"

resnet_backbone = resnet20(num_classes=1)
model = SimCLRModel(resnet_backbone, img_size = input_size)
trainer = pl.Trainer(
    max_epochs=50, gpus=gpus, progress_bar_refresh_rate=10
)
trainer.fit(model, dataloader_train)

/trinity/home/d.volkov/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=10)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: /trinity/home/d.volkov/proj18/examples/lightning_logs

  | Name            | Type                 | Params
---------------------------------------------------------
0 | backbone        | Sequential           | 269 K 
1 | projection_head | SimCLRPr

In [17]:
torch.save(model.state_dict(), '../weights/weights_simclr')

# 2. Embeddings extraction
Next we create a helper function to generate embeddings
from our test images using the model we just trained.
Note that only the backbone is needed to generate embeddings,
the projection head is only required for the training.
Make sure to put the model into eval mode for this part!



In [13]:
model.to(device)
model.eval()
embeddings_test, filenames_test = generate_embeddings(model, dataloader_test, device)
embeddings_train, filenames_train = generate_embeddings(model, dataloader_train, device)

In [14]:
print(f'Shape of TEST embeddings {embeddings_test.shape}')
print(f'Shape of TRAIN embeddings {embeddings_train.shape}')

Shape of TEST embeddings (5000, 64)
Shape of TRAIN embeddings (4992, 64)


# 3. Calculate Hausdorff distance between poin clouds

In [15]:
from scipy.spatial.distance import directed_hausdorff

hausdorff_dist = directed_hausdorff(embeddings_train, embeddings_test)[0]

print(f'Hausdorff Dist: {hausdorff_dist:.3f}')

Hausdorff Dist: 0.371
